In [1]:
source("../wwa_model-fitting.R")
source("../00_model-fitting-dev/multivariate/model-fitting_multi-covariate.R")

## **Additional model fitting function**

In [2]:
cmodel_results <- function(mdl, rp = 10, cov_now, cov_hist, cov_fut, nsamp = 1000, seed = 42, di_relative = NA, y_start = 1979, y_now = 2023, y_fut = 2050) {
    
    set.seed(seed)
    
    # fill in missing parameters
    if(is.na(di_relative)) di_relative <- mdl$type == "fixeddisp"
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # FIT MODEL to three different subsets: for evaluation, attribution & projection
    
    df <- mdl$data
    
    # fit models
    mdl_eval <- refit(mdl, new_df = df[df$year >= y_start & df$year <= y_now,])
    mdl_attr <- refit(mdl, new_df = df[df$year <= y_now,])
    mdl_proj <- refit(mdl, new_df = df[df$year <= y_fut,])
    
    # get return level to use for analysis
    event_rl <- eff_return_level(mdl_attr, rp = rp, fixed_cov = cov_now)
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ## Bootstrap each set of model results
    
    if(di_relative) { di_cnm <- "dI_rel" } else { di_cnm <- "dI_abs" }
    if(mdl$type == "fixeddisp") { key_par <- "disp" } else { key_par <- "sigma0" }
    if(mdl$dist == "gev") { key_par <- c(key_par, "shape") }
    
    ci_eval <- boot_ci(mdl_eval, cov = cov_now, cov_cf = cov_hist, ev = event_rl, rp = rp, nsamp = nsamp)[c(key_par),,drop = F]
    ci_attr <- boot_ci(mdl_attr, cov = cov_now, cov_cf = cov_hist, ev = event_rl, rp = rp, nsamp = nsamp)[c("PR", di_cnm),]
    ci_proj <- boot_ci(mdl_proj, cov = cov_now, cov_cf = cov_fut, ev = event_rl, rp = rp, nsamp = nsamp)[c("PR", di_cnm),]
                       
    # invert future projections
    ci_proj["PR",] <- 1/ci_proj["PR",c(1,3,2)]
    ci_proj[di_cnm,] <- -ci_proj[di_cnm, c(1,3,2)]
    
    # flatten & rename
    ci_eval <- unlist(lapply(rownames(ci_eval), function(cnm) setNames(ci_eval[cnm,], paste("eval", gsub("_", "-", cnm), c("est", "lower", "upper"), sep = "_"))))
    ci_attr <- unlist(lapply(rownames(ci_attr), function(cnm) setNames(ci_attr[cnm,], paste("attr", gsub("_", "-", cnm), c("est", "lower", "upper"), sep = "_"))))
    ci_proj <- unlist(lapply(rownames(ci_proj), function(cnm) setNames(ci_proj[cnm,], paste("proj", gsub("_", "-", cnm), c("est", "lower", "upper"), sep = "_"))))
                         
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    res <- t(data.frame(c(ci_eval, "rp_value" = event_rl, ci_attr, ci_proj)))     
    rownames(res) <- paste0(mdl$varnm, " ~ ", paste0(mdl$covnm, collapse = " + "), " (rp", rp,")")
    return(res)
}

## **Example**

In [3]:
df <- read.csv("df.csv")

In [4]:
# fit basic form of model
mdl <- fit_ns("gev", "shift", data = df, varnm = "var", covnm = c("gmst", "nino"))

y_now <- 2024

cov_2024 <- df[df$year == y_now,c("gmst", "nino"),drop = F]
cov_hist <- df[df$year == y_now,c("gmst", "nino"),drop = F] - c(1.2,0)
cov_fut <- df[df$year == y_now,c("gmst", "nino"),drop = F] + c(0.8,0)

In [5]:
cmodel_results(mdl, rp = 30, cov_now = cov_2024, cov_hist = cov_hist, cov_fut = cov_fut, y_now = 2024)

,eval_sigma0_est,eval_sigma0_lower,eval_sigma0_upper,eval_shape_est,eval_shape_lower,eval_shape_upper,rp_value,attr_PR_est,attr_PR_lower,attr_PR_upper,attr_dI-abs_est,attr_dI-abs_lower,attr_dI-abs_upper,proj_PR_est,proj_PR_lower,proj_PR_upper,proj_dI-abs_est,proj_dI-abs_lower,proj_dI-abs_upper
var ~ gmst + nino (rp30),45.30121,30.76185,59.95657,0.1201768,-0.4568368,0.5024565,382.4078,1.19205,0.7972616,2.11497,14.57023,-15.57928,45.11507,1.28089,1.050817,2.069517,15.58985,3.61575,30.01042
